In [1]:
import pandas as pd

# Journees_europeennes_du_patrimoine_20160914.csv

In [15]:
data = pd.read_csv('Journees_europeennes_du_patrimoine_20160914.csv', delimiter=';', low_memory=False)
data.tail()

,Identifiant,Titre - FR,Titre - DE,Titre - EN,Titre - ES,Titre - IT,Titre - BR,Titre - PL,Titre - CA,Titre - OC,...,Accès / Itinéraire - DE,Accès / Itinéraire - EN,Accès / Itinéraire - ES,Accès / Itinéraire - IT,Accès / Itinéraire - BR,Accès / Itinéraire - PL,Accès / Itinéraire - CA,Accès / Itinéraire - OC,Accès / Itinéraire - ZH,Accès / Itinéraire - JA
23619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data.columns

Index(['Identifiant', 'Titre - FR', 'Titre - DE', 'Titre - EN', 'Titre - ES',
       'Titre - IT', 'Titre - BR', 'Titre - PL', 'Titre - CA', 'Titre - OC',
       ...
       'Accès / Itinéraire - DE', 'Accès / Itinéraire - EN',
       'Accès / Itinéraire - ES', 'Accès / Itinéraire - IT',
       'Accès / Itinéraire - BR', 'Accès / Itinéraire - PL',
       'Accès / Itinéraire - CA', 'Accès / Itinéraire - OC',
       'Accès / Itinéraire - ZH', 'Accès / Itinéraire - JA'],
      dtype='object', length=122)

In [34]:
print(f"Initial number of rows: {data.shape[0]}")


Initial number of rows: 23624


## Data Cleaning

### Dropping Irrelevant Columns

In [97]:
columns_to_drop = [col for col in data.columns if '- DE' in col or '- ES' in col or '- IT' in col or 
                   '- BR' in col or '- PL' in col or '- CA' in col or '- OC' in col or '- ZH' in col or '- JA' in col or '- EN' in col]
cleaned_data = data.drop(columns=columns_to_drop)

In [98]:
seuil = len(cleaned_data) * 0.2
cleaned_data = cleaned_data.dropna(axis=1, thresh=seuil)

In [99]:
cleaned_data.columns

Index(['Identifiant', 'Titre - FR', 'Description - FR',
       'Description longue - FR', 'Description longue - HTML - FR',
       'Dernière mise à jour', 'Résumé horaires - FR',
       'Horaires détaillés - FR', 'Horaires ISO', 'Première date',
       'Première ouverture', 'Première fermeture', 'Type d'événement',
       'Conditions tarifaires', 'Conditions de participation', 'Age maximum',
       'Identifiant du lieu', 'Latitude', 'Longitude', 'Nom du lieu',
       'Adresse', 'Code postal', 'Ville', 'Département', 'Région', 'Pays',
       'Site web du lieu', 'Tags du lieu', 'Description du lieu - FR'],
      dtype='object')

In [100]:
print(f"Number of rows after dropping irrelevant columns: {cleaned_data.shape[0]}")

Number of rows after dropping irrelevant columns: 23624


### Remove duplicate rows

In [101]:
cleaned_data = cleaned_data.drop_duplicates()
print(f"Number of rows after removing duplicates: {cleaned_data.shape[0]}")

Number of rows after removing duplicates: 22981


### Fill missing values in categorical columns

In [102]:
missing_values_before = cleaned_data.isnull().sum()

In [103]:
for column in cleaned_data.select_dtypes(include='object').columns:
    cleaned_data[column].fillna('Unknown', inplace=True)

In [104]:
missing_values_after = cleaned_data.isnull().sum()
filled_columns = missing_values_before[missing_values_before > 0] - missing_values_after
filled_columns = filled_columns[filled_columns > 0]
print("\nColumns where missing values were filled:")
print(filled_columns)


Columns where missing values were filled:
Titre - FR                        10686
Description - FR                  13395
Description longue - FR           17877
Description longue - HTML - FR    17877
Dernière mise à jour              14972
Résumé horaires - FR               7663
Horaires détaillés - FR           14785
Horaires ISO                      14840
Première date                      4742
Première ouverture                   55
Première fermeture                  183
Type d'événement                  17914
Conditions tarifaires             17890
Conditions de participation       17989
Nom du lieu                       11156
Adresse                           12920
Ville                              6426
Département                          92
Région                               31
Pays                                  1
Site web du lieu                  15551
Tags du lieu                       4175
Description du lieu - FR          15160
dtype: int64


### Drop rows where the Title is unknowed

In [108]:
cleaned_data = cleaned_data[cleaned_data['Titre - FR'] != 'Unknown']

In [111]:
print(f"Number of rows after removing non existent title: {cleaned_data.shape[0]}")

Number of rows after removing non existent title: 12295


### Convert Float into Int

In [115]:
for col in ['Identifiant', 'Age maximum', 'Identifiant du lieu', 'Code postal']:
    cleaned_data[col] = cleaned_data[col].astype('Int64')

In [116]:
cleaned_data.dtypes

Identifiant                         Int64
Titre - FR                         object
Description - FR                   object
Description longue - FR            object
Description longue - HTML - FR     object
Dernière mise à jour               object
Résumé horaires - FR               object
Horaires détaillés - FR            object
Horaires ISO                       object
Première date                      object
Première ouverture                 object
Première fermeture                 object
Type d'événement                   object
Conditions tarifaires              object
Conditions de participation        object
Age maximum                         Int64
Identifiant du lieu                 Int64
Latitude                          float64
Longitude                         float64
Nom du lieu                        object
Adresse                            object
Code postal                         Int64
Ville                              object
Département                       

## Final result

In [117]:
cleaned_data.head()

,Identifiant,Titre - FR,Description - FR,Description longue - FR,Description longue - HTML - FR,Dernière mise à jour,Résumé horaires - FR,Horaires détaillés - FR,Horaires ISO,Première date,...,Nom du lieu,Adresse,Code postal,Ville,Département,Région,Pays,Site web du lieu,Tags du lieu,Description du lieu - FR
0,75686965,La citoyenneté,Une exposition de l'office national des ancien...,Une exposition grand public mais accessible au...,<p>Une exposition grand public mais accessible...,2016-08-31T10:07:47.000Z,"16 septembre, 08h00",vendredi 16 septembre - 08h00 à 17h00,2016-09-16T08:00:00+04:00-2016-09-16T17:00:00+...,2016-09-16,...,Hôtel de ville de Saint-Pierre,"rue Méziaire Guignard, 97416 Saint-Pierre",<NA>,Unknown,La Réunion,La Réunion,Réunion,Unknown,"Monument historique|Lieu de pouvoir, édifice j...","Ancien entrepôt de la Compagnie des Indes, ce ..."
1,18268067,Journée du patrimoine - St-Aignan,Événement,Événement#JEP2016* Programmation exceptionne...,<p>Événement</p><p>#JEP2016</p><ul><li>Program...,2016-09-07T08:16:59.000Z,16 et 17 septembre,vendredi 16 septembre - 07h00 à 08h00vendredi ...,2016-09-16T07:00:00+02:00-2016-09-16T08:00:00+...,2016-09-16,...,Saint-Aignan,1rue Victor-Hugo 41110 Saint-Aignan-sur-Cher,41110,Saint-Aignan-sur-Cher,Loir-et-Cher,Centre-Val de Loire,France,https://www.facebook.com/MairieStAignan/,Patrimoine XXe|Patrimoine européen|Villes et P...,Événement #JEP2016- Programmation exceptionnel...
2,95583734,Du côté de la Saline-les-Hauts avec l'associat...,Découverte du Jako Malbar,"L’association ""Kaz Maron"" nous entraînera dans...",<p>L’association &quot;Kaz Maron&quot; nous en...,2016-07-28T04:22:16.000Z,16 et 17 septembre,vendredi 16 septembre - 09h00 à 11h30vendredi ...,2016-09-16T09:00:00+04:00-2016-09-16T11:30:00+...,2016-09-16,...,Ancienne usine de Vue-Belle,"rue du lycée, 97422 la Saline-les-Hauts",97411,La Saline Les Hauts,La Réunion,La Réunion,Réunion,Unknown,Unknown,Unknown
3,57016244,Raconte-moi le Patrimoine,Et si vous preniez la parole pour les Journées...,Et si vous preniez la parole pour les Journées...,<p>Et si vous preniez la parole pour les Journ...,2016-07-12T10:09:26.000Z,16 - 18 septembre,vendredi 16 septembre - 07h00 à 06h00samedi 17...,2016-09-16T07:00:00+02:00-2016-09-17T06:00:00+...,2016-09-16,...,Pays voironnais (34 communes),38500 Voiron,38500,Voiron,Isere,Auvergne-Rhône-Alpes,France,Unknown,Unknown,Unknown
4,3120479,"Randonnée pédestre ""Au fil du temps""",Mercredi 14 sept à 15h,Nous vous proposerons un itinéraire qui vous m...,<p>Nous vous proposerons un itinéraire qui vou...,2016-08-23T13:34:19.000Z,"16 septembre, 07h00",vendredi 16 septembre - 07h00 à 08h00,2016-09-16T07:00:00+02:00-2016-09-16T08:00:00+...,2016-09-16,...,Saint-Paul-lès-Dax,"Bourg, 40990 Saint-Paul-lès-Dax",40990,Saint-Paul-lès-Dax,Landes,Aquitaine-Limousin-Poitou-Charentes,France,http://www.st-paul-les-dax.fr,Unknown,Église©Ghislain118


In [118]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12295 entries, 0 to 23614
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Identifiant                     7351 non-null   Int64  
 1   Titre - FR                      12295 non-null  object 
 2   Description - FR                12295 non-null  object 
 3   Description longue - FR         12295 non-null  object 
 4   Description longue - HTML - FR  12295 non-null  object 
 5   Dernière mise à jour            12295 non-null  object 
 6   Résumé horaires - FR            12295 non-null  object 
 7   Horaires détaillés - FR         12295 non-null  object 
 8   Horaires ISO                    12295 non-null  object 
 9   Première date                   12295 non-null  object 
 10  Première ouverture              12295 non-null  object 
 11  Première fermeture              12295 non-null  object 
 12  Type d'événement                12295

In [119]:
cleaned_data.columns

Index(['Identifiant', 'Titre - FR', 'Description - FR',
       'Description longue - FR', 'Description longue - HTML - FR',
       'Dernière mise à jour', 'Résumé horaires - FR',
       'Horaires détaillés - FR', 'Horaires ISO', 'Première date',
       'Première ouverture', 'Première fermeture', 'Type d'événement',
       'Conditions tarifaires', 'Conditions de participation', 'Age maximum',
       'Identifiant du lieu', 'Latitude', 'Longitude', 'Nom du lieu',
       'Adresse', 'Code postal', 'Ville', 'Département', 'Région', 'Pays',
       'Site web du lieu', 'Tags du lieu', 'Description du lieu - FR'],
      dtype='object')

## Export Data

In [120]:
cleaned_data.to_csv('Journees_europeennes_du_patrimoine_20160914_cleaned.csv', index=False, encoding='utf-8')